In [1]:
import logging

from bs4 import BeautifulSoup, SoupStrainer, Tag
from urllib.request import urlopen
import urllib
import pandas as pd
import time
from urllib.request import Request, urlopen

## Per Page Recipe Scraping

In [ ]:
url =  \
"https://www.tarladalal.com/sabudana-pakoras-14757r"
# "https://www.tarladalal.com/chakli-instant-chakli-40487r"
# "https://www.tarladalal.com/pressure-cooker-oondhiya-4337r"
# "https://www.tarladalal.com/bajra-dhebra-recipe-gujarati-tea-time-snack-42272r"
# "https://www.tarladalal.com/sabudana-khichdi-in-microwave-2749r"
# "https://www.tarladalal.com/sweet-shakarpara-baked-sweet-shakar-para-41973r"
# "https://www.tarladalal.com/vegetable-stuffed-upma-cutlet-36118r"
# "https://www.tarladalal.com/surti-undhiyu-40596r"
# "https://www.tarladalal.com/cauliflower-nu-bhanolu--gujarati-recipe-560r"
# "https://www.tarladalal.com/instant-lemon-pickle--zero-oil-22184r"
# "https://www.tarladalal.com/sprouts-spring-onion-and-tomato-salad-2957r" 
# "https://www.tarladalal.com/khandvi-gujarati-snack-recipe-557r" 
# "https://www.tarladalal.com/sabudana-khichdi-in-microwave-2749r"
hdr = {'User-Agent':'Mozilla/5.0'}
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
req = urllib.request.Request(url=url, headers=header) 
page = urllib.request.urlopen(req).read()
soup = BeautifulSoup(page.decode("utf-8", "html.parser"))

webpage = urlopen(req).read()
soup = BeautifulSoup(webpage.decode("utf-8", "html.parser"))

In [ ]:
with open("/home/shalin/t.html", 'w') as f:
    f.write(webpage)

In [ ]:
# ing_sections = soup.find_all('span', class_='recipe_subheader', id=)
# for i in ing_sections:
#     print(i)
#     print("------------")
ing_sections = soup.find_all('span', id=lambda x: x and x.startswith('ingsection'))

recipes_ingredients = ""
# Iterate over each 'ingsection' and print its content
for section in ing_sections:
    if not recipes_ingredients:
        recipes_ingredients += section.text.strip() + ":"
    else:
        recipes_ingredients += ";" + section.text.strip() + ":"
    next_sibling = section.find_next_sibling()  # Get the next sibling
    siblings = []
    while next_sibling:
        if next_sibling.name == 'span' and next_sibling.has_attr('class') and 'recipe_subheader' in next_sibling['class']:
            break  # Break if next sibling is another 'ingsection'
        if next_sibling.text.strip():
            # print(next_sibling.text.strip())  # Print the content
            siblings.append(next_sibling.text.strip())
        next_sibling = next_sibling.find_next_sibling()  # Move to the next sibling
    recipes_ingredients += ", ".join(siblings)
recipes_ingredients

In [ ]:
soup.find('span', id='ctl00_cntrightpanel_lblrecipeNameH2').text

In [ ]:
# TODO: For https://www.tarladalal.com/vegetable-stuffed-upma-cutlet-36118r, there's section in the recipe steps. Needs sections in the recipes too
recipe_small_steps_div = soup.find('div', id='recipe_small_steps')
# recipe_small_steps_div.find_all('span')
for span in recipe_small_steps_div.find_all('li'):
    print(span.get_text())

In [ ]:
recipe_small_steps_subheaders = soup.find_all(class_='recipe_subheader')
recipe_small_steps_subheaders

In [ ]:
print(get_ingrediants(soup, url))

In [ ]:
rdiv = soup.find('div', id="rcpinglist")
for sp in rdiv.select('span'):
    if sp.has_attr('itemprop'):
        print(sp.text.strip())
    elif sp.has_attr('class') and 'recipe_subheader' in sp['class']:
        print(sp.text.strip() + ":")

In [ ]:
rec = soup.find('div', id="recipe_small_steps").get_text("\n").strip()
print(rec)
print()
print("\n".join(list(dict.fromkeys(rec.split("\n")))))

In [ ]:
recipe_small_steps_subheaders = soup.find_all(class_='recipe_subheader')
steps = []
for subheader in recipe_small_steps_subheaders:
    if subheader.has_attr('id') and subheader['id'].startswith('procsection'):
        # print(subheader)
        print(subheader.text.strip())  # Print subheader text
        # Find the parent ol element
        parent_ol = subheader.find_next('ol')
        # Find all li elements under the parent ol
        recipe_steps = parent_ol.find_all('li')
        # Extract and print the text for each step
        for step in recipe_steps:
            print(step.span.text.strip())
            steps.append(step.span.text.strip())
if not steps:
    recipe_small_steps_div = soup.find('div', id='recipe_small_steps')
    # recipe_small_steps_div.find_all('span')
    for span in recipe_small_steps_div.find_all('li'):
        print(span.get_text())
        steps.append(span.get_text())
steps

In [ ]:
parent_span = soup.find('span', id='ctl00_cntrightpanel_lblServes')
# text_in_parent_span = ''.join([str(x) for x in parent_span.contents if isinstance(x, str)])
print(text_in_parent_span)

In [ ]:
time_tag = soup.find('time', itemprop='totalTime')
print(time_tag)
span_tag = time_tag.find('span')
if span_tag:
    span_tag.decompose()
print(time_tag.text)

In [ ]:
soaking_time_element = soup.find(text=lambda text: 'Soaking time' in text)
print(soaking_time_element.split(':')[1].split('\xa0')[1])
# Extract the soaking time value
if soaking_time_element:
    soaking_time = soaking_time_element.split(':')[1].split('\xa0')[1].strip()
    print("Soaking time:", soaking_time)
else:
    print("Soaking time not found")

In [ ]:
soup.find('span', id='ctl00_cntrightpanel_lblrecipeNameH2').text

In [ ]:
print(soup.find('div', id='recipe_tags').get_text("\n").strip())

In [ ]:
get_time_and_serving(soup, url)

In [ ]:
tags_div = soup.find('div', id='recipe_tags')
p_tag = tags_div.find_next('p')
for t in p_tag.find_all('time'):
    t.decompose()
for serv_time in p_tag.find_all('span', attrs=dict(id="ctl00_cntrightpanel_lblServes")):
    serv_time.decompose()
print(p_tag.get_text().
      replace("Preparation Time:", "").
      replace("Cooking Time:", "").
      replace("Total Time:", "").
      replace("&nbsp", "").
      split("Show me for")[0].strip())

## Page Links Scraping

In [ ]:
links_url = "https://www.tarladalal.com/recipes-for-veg-recipes-gujarati-24?pageindex=1" #"https://www.tarladalal.com/sprouts-spring-onion-and-tomato-salad-2957r" # "https://www.tarladalal.com/sabudana-khichdi-in-microwave-2749r" #
links_header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
links_req = urllib.request.Request(url=links_url, headers=links_header) 
links_page = urllib.request.urlopen(links_req).read()

In [ ]:
links_soup = BeautifulSoup(links_page.decode("utf-8", "html.parser"))

In [ ]:
links_soup

In [ ]:
for div in links_soup.findAll('div', class_="rcc_rcpcore"):
    print(div)
    print("**********************************")

In [ ]:
for sp in links_soup.find_all('span', attrs={"class":"rcc_recipename"}):
    print(sp.text)
    print(sp.find('a', href=True)['href'])

### Final Scrapper

In [2]:
def get_webpage(links_url):
    try:
        links_header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
          'AppleWebKit/537.36 (KHTML, like Gecko) '
          'Chrome/123.0.0.0 Safari/537.36',
          'Accept': '*/*', #'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.9',
          'Connection': 'keep-alive',
        }
        links_req = urllib.request.Request(url=links_url, headers=links_header) 
        try:
            links_page = urllib.request.urlopen(links_req, timeout=90).read()
            # print(links_page)
        except urllib.error.HTTPError:
            logging.exception(f"Error when getting webpage from {links_url}")
            time.sleep(120)
            logging.info("Retrying......")
            try:
                links_page = urllib.request.urlopen(links_req, timeout=90).read()
            except:
                logging.warning("Retry Failed!!!")
                return 
        links_soup = BeautifulSoup(links_page.decode("utf-8", "html.parser"))
        return links_soup
        
        # req = Request(
        #     url=url, 
        #     headers={'User-Agent': 'Mozilla/5.0'}
        # )
        # webpage = urlopen(req).read()
        # links_soup = BeautifulSoup(webpage.decode("utf-8", "html.parser"))
        # return links_soup
    except:
        logging.exception(f"Error when getting webpage code for {links_url}")

In [3]:
def get_ingrediants(soup, page_link):
    recipes_ingredients = ""
    try:
        ing_list = []
        rdiv = soup.find('div', id="rcpinglist")
        for sp in rdiv.select('span'):
            if sp.has_attr('itemprop'):
                ing_list.append(sp.text.strip())
            elif sp.has_attr('class') and 'recipe_subheader' in sp['class']:
                ing_list.append(sp.text.strip() + ":")
        recipes_ingredients = "\n ".join(ing_list)
    except:
        logging.exception(f"Exception when finding ingrediants of {page_link}")
    
    if not recipes_ingredients:
        logging.warning(f"Cannot find recipe ingredients for {page_link}")
    return recipes_ingredients        

In [4]:
def get_recipe(soup, page_link):
    recipe = ""
    try:
        rec = soup.find('div', id="recipe_small_steps").get_text("\n").strip()
        recipe = "\n".join(list(dict.fromkeys(rec.split("\n"))))
    except:
        logging.exception(f"Exception when finding recipe for {page_link}")

    if not recipe:
        logging.warning(f"Cannot find recipe method for {page_link}")
    return recipe

In [5]:
def get_time_and_serving(soup, page_link):
    times = ""
    servings = ""
    other_info = ""
    try:
        prep_time = ""
        cook_time = ""
        total_tie = ""
        
        servings_tag = soup.find('span', attrs=dict(itemprop="recipeYield"))
        if servings_tag:
            servings = str(servings_tag.text)
        
        prep_tag = soup.find('time', attrs=dict(itemprop="prepTime"))
        if prep_tag:
            prep_time = prep_tag.text
            times += "Preparation Time: " + prep_time+ ", "

        cook_tag = soup.find('time', attrs=dict(itemprop="cookTime"))
        if cook_tag:
            cook_time = cook_tag.text
            times += "Cooking Time: " + cook_time + ", "
        
        total_time_tag = soup.find('time', itemprop='totalTime')
        if total_time_tag:
            span_tag = total_time_tag.find('span')
            if span_tag:
                span_tag.decompose()
            total_time = total_time_tag.text
            times += "Total Time: " + total_time + ", "
  
        # Other time or coocking related info
        tags_div = soup.find('div', id='recipe_tags')
        if not tags_div:
            tags_div = soup.find('div', id='ctl00_cntrightpanel_lblrecipeNameH2')
            
        p_tag = tags_div.find_next('p')
        for t in p_tag.find_all('time'):
            t.decompose()
        if servings:
            for serv_time in p_tag.find_all('span', attrs=dict(id="ctl00_cntrightpanel_lblServes")):
                serv_time.decompose()
        
        other_info = p_tag.get_text().replace("&nbsp", " ").split("Show me for")[0].strip()        
        if prep_time:
              other_info = other_info.replace("Preparation Time:", "")
        if cook_time:
              other_info = other_info.replace("Cooking Time:", "")
        if total_time:
              other_info = other_info.replace("Total Time:", "")
        
        times += other_info
    
    except:
        logging.exception(f"Error while finding time and serving for {page_link}") 

    if not times or not servings:
        logging.warning(f"Cannot find times, or servings for {page_link}")
    return times, servings

#### Get the pages and content

In [39]:
recipe_list_page_url = "https://www.tarladalal.com/recipes-for-veg-recipes-gujarati-24?pageindex={0}"
recipe_url_prefix = "https://www.tarladalal.com/"
for recipe_list_page_idx in range(1, 2):
    recipe_list = []
    print(f"Getting recipies from page {recipe_list_page_idx}")
    list_url = recipe_list_page_url.format(recipe_list_page_idx)
    recipe_list_page = get_webpage(list_url)
    if not recipe_list_page:
        continue
    for recipe_span in recipe_list_page.find_all('span', attrs={"class":"rcc_recipename"}):
        df_row = {}
        df_row["recipe_list_url"] = list_url
        df_row["recipe_name"] = recipe_span.text
        recipe_url = recipe_url_prefix + recipe_span.find('a', href=True)['href']
        df_row["recipe_url"] = recipe_url    
        # print(recipe_url)
        recipe_page = get_webpage(recipe_url)
        if not recipe_page:
            logging.warn(f"Couldn't find recipe page for {recipe_url}")
            continue
        df_row["recipe_long_name"] = recipe_page.find('span', id='ctl00_cntrightpanel_lblrecipeNameH2').text
        df_row["recipe_ing"] = get_ingrediants(recipe_page, recipe_url)
        df_row["recipe"] = get_recipe(recipe_page, recipe_url)
        df_row["tags"] = recipe_page.find('div', id='recipe_tags').get_text("\n").strip()
        if not df_row["tags"]:
            logging.warn(f"Couldn't find tags for {recipe_url}")
        times, servings = get_time_and_serving(recipe_page, recipe_url)
        df_row["times"] = times
        df_row["servings"] = servings
        recipe_list.append(df_row)
    recipe_df = pd.DataFrame(recipe_list)
    recipe_df.to_csv(f"../data/recipe_page_{recipe_list_page_idx}.csv", index=False)

ERROR:root:Error when getting webpage from https://www.tarladalal.com/recipes-for-veg-recipes-gujarati-24?pageindex=1
Traceback (most recent call last):
  File "/tmp/ipykernel_7318/3896314710.py", line 14, in get_webpage
    links_page = urllib.request.urlopen(links_req, timeout=90).read()
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/shalin/anaconda3/envs/genai/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/shalin/anaconda3/envs/genai/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/home/shalin/anaconda3/envs/genai/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "/home/shalin/anaconda3/envs/genai/lib/python3.11/urllib/request.py", line 563, in error
    return self._call_chain(*

Getting recipies from page 1


#### TODO:
Page 1: Need other times in addition of cooking etc time.

All Pages, 
- last line of recipe misses item name
- Remove "Tags" from tags column

In [33]:
for page_id in range(2, 39):
    df = pd.read_csv(f"../data/recipe_page_{page_id}.csv")
    print(f"Page Idx: {page_id}")
    for idx, row in df.iterrows():
        if len(df.loc[idx, "recipe_ing"].split("\n")) <= 3:
            print(f'Less ing for {row["recipe_url"]}')
    for idx, row in df.iterrows():
        if len(df.loc[idx, "recipe"].split("\n")) <= 3:
            print(f'Less recipe for {row["recipe_url"]}')

Page Idx: 2
Less ing for https://www.tarladalal.com/how-to-make-rajgira-flour-amaranth-flour-at-home-42738r
Page Idx: 3
Page Idx: 4
Less ing for https://www.tarladalal.com/phulka-recipe-indian-chapati-recipe-4392r
Less recipe for https://www.tarladalal.com/mango-milkshake-38492r
Page Idx: 5
Page Idx: 6
Page Idx: 7
Page Idx: 8
Less ing for https://www.tarladalal.com/daria-chikki-roasted-chana-dal-chikki-40998r
Less ing for https://www.tarladalal.com/alphonso-aamras-aaamras-recipe-40815r
Page Idx: 9
Page Idx: 10
Page Idx: 11
Page Idx: 12
Page Idx: 13
Less ing for https://www.tarladalal.com/peanut-chikki-41000r
Less recipe for https://www.tarladalal.com/garlic-chutney--gujarati-recipe-573r
Less recipe for https://www.tarladalal.com/onion-and-raw-mango-chunda--kaanda-aur-kairi-ka-choonda-40887r
Less recipe for https://www.tarladalal.com/ukado-gujarati-lemon-grass-and-ginger-milk-41739r
Page Idx: 14
Page Idx: 15
Less ing for https://www.tarladalal.com/coconut-chikki-kopra-chikki-made-in-jag

In [38]:
s = """ <br/><span itemprop="recipeIngredient"><span>1</span>  vati <a href="glossary-chana-dal-split-bengal-gram-285i"><span>chana dal (split Bengal gram)</span></a></span><br/><span itemprop="recipeIngredient"><span>1/2</span>  vati <a href="glossary-jaggery-gur-gud-kala-gud-477i"><span>jaggery (gur)</span></a></span><br/><span itemprop="recipeIngredient"><span>1/2</span>  vati <a href="glossary-sugar-chini-shakkar-278i"><span>sugar</span></a></span><br/>10-12 veldodyanchi pud,<br/>thodese Keshar,<br/>1/4 jayfal pud,<br/>1 chamcha rava(aadhi thoda bhijat ghalava),<br/>1 vati kanik,<br/>1 chamcha tel,<br/>mith<br/>
"""
sp = BeautifulSoup(s)
print(sp.get_text("\n"))

1
  vati 
chana dal (split Bengal gram)
1/2
  vati 
jaggery (gur)
1/2
  vati 
sugar
10-12 veldodyanchi pud,
thodese Keshar,
1/4 jayfal pud,
1 chamcha rava(aadhi thoda bhijat ghalava),
1 vati kanik,
1 chamcha tel,
mith




In [47]:
# Remove tags from tags column
for page_id in range(2, 39):
    df = pd.read_csv(f"../data/recipe_page_{page_id}.csv")
    print(f"Page Idx: {page_id}")
    # for idx, row in df.iterrows():
    df['tags'] = df['tags'].str.replace("Tags", "").str.strip()
    df.to_csv(f"../data/recipe_page_{page_id}.csv")

Page Idx: 2
Page Idx: 3
Page Idx: 4
Page Idx: 5
Page Idx: 6
Page Idx: 7
Page Idx: 8
Page Idx: 9
Page Idx: 10
Page Idx: 11
Page Idx: 12
Page Idx: 13
Page Idx: 14
Page Idx: 15
Page Idx: 16
Page Idx: 17
Page Idx: 18
Page Idx: 19
Page Idx: 20
Page Idx: 21
Page Idx: 22
Page Idx: 23
Page Idx: 24
Page Idx: 25
Page Idx: 26
Page Idx: 27
Page Idx: 28
Page Idx: 29
Page Idx: 30
Page Idx: 31
Page Idx: 32
Page Idx: 33
Page Idx: 34
Page Idx: 35
Page Idx: 36
Page Idx: 37
Page Idx: 38
